<a href="https://colab.research.google.com/github/iamkaikai/Numerai/blob/main/Numerai_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup the environment

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd ./gdrive/MyDrive/Numerai
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
# !cat /proc/cpuinfo
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Numerai
Sun Mar 26 21:04:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                            

In [2]:
!pip install -U pip && pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.4/899.4 kB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB

In [3]:
import matplotlib
import gc
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import random
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import joblib
import math
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

In [4]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import StepLR
from collections import Counter
# from numerapi import NumerAPI
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ARDRegression
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from utils import save_model, load_model, neutralize, get_biggest_change_features, validation_metrics
# from sklearn import (
#     feature_extraction, feature_selection, decomposition, linear_model,
#     model_selection, metrics, svm
# )
from utils import (
    save_model, load_model, neutralize,
    get_biggest_change_features,
    get_time_series_cross_val_splits,
    validation_metrics,
    ERA_COL, DATA_TYPE_COL, TARGET_COL, EXAMPLE_PREDS_COL
)

# Downlaod the dataset via Numerai.api

In [ ]:
# napi = NumerAPI()
# print("Downloading dataset files...")
# dataset_name = "v4"
# current_round = napi.get_current_round()
# napi.download_dataset("{dataset_name}/train.parquet", "train.parquet")
# napi.download_dataset("{dataset_name}/validation.parquet", "validation.parquet")
# napi.download_dataset("{dataset_name}/live.parquet", "live.parquet")
# napi.download_dataset("{dataset_name}/live_example_preds.parquet", "live_example_preds.parquet")
# napi.download_dataset("{dataset_name}/validation_example_preds.parquet", "validation_example_preds.parquet")

# Some functions for later usages

In [5]:
def prediction_clip(read_file, save_name):
    read = pd.read_csv(read_file)
    read.loc[read['prediction'] < 0, 'prediction'] = 0
    read.loc[read['prediction'] > 1, 'prediction'] = 1
    read.to_csv(read_file, index=False)

#input np.array of the prediction from the model
def valid_gen_and_save(id, prediction,version_name):
    data = {'id': id, 'prediction': np.round(prediction, 8)}
    pred_file = pd.DataFrame(data)
    current_round = 433
    file_name = f"./prediction/val_{current_round}_{version_name}.csv"
    pred_file.to_csv(file_name, index=False)
    prediction_clip(file_name, file_name)

def read():
    print('init df_train...')
    df_train = pd.read_parquet('./v4/train.parquet')
    df_train.info()  
    
    features = [c for c in df_train if c.startswith("feature")]
    targets = [c for c in df_train if c.startswith("target")]
    targets_60 = [c for c in df_train if (c.startswith("target") and c.endswith("60"))]
    eras_features_and_targets = [c for c in df_train if (c.startswith("target") or c.startswith("feature") or c.startswith("era"))]
    feature_size = len(features)
    target_size = len(targets)
    eras_features_and_targets_size = len(eras_features_and_targets)

    print("The size of the feature is:", feature_size)
    print("The size of the targets is:", target_size)
    print("The size of eras + features + targets =", eras_features_and_targets_size)
    # X_train = df_train
    df_train['era'] = df_train['era'].astype(int)
    eras = df_train['era'].unique()
    eras_len = len(eras)
    
    return df_train, eras_len

def get_targets(top_num):
    with open("./corr/good_features.pkl", 'rb') as f:
        good_features = pickle.load(f)

    ult_good_features = []
    for era in good_features:
        for feature in era:
            ult_good_features.append(feature)
        
    count_counter = Counter(ult_good_features)
    ult_good_features = list(count_counter.most_common())

    desire_features = []
    for i in range(top_num):
        desire_features.append(ult_good_features[i][0])
        
    return desire_features

In [6]:
df_train, eras_len = read()     #init df_train
# features = get_targets(700)     #get top 700 features
features = [c for c in df_train if c.startswith("feature")]         #get all 1191 feature
targets = [c for c in df_train if c.startswith("target")]       #get all targets
targets.remove('target')
# targets = ['target_nomi_v4_20','target_nomi_v4_60']
# targets = ['target']
print(targets)
print(f"num of target: {len(targets)}")
X = df_train[features].fillna(0.5).values
y = df_train[targets].fillna(0.5).values
print('Training data filled finished!!!!!!!!!!!!!!!')
del df_train
print('df_train cleaned!!!!!!!!!!!!!!!')

init df_train...
<class 'pandas.core.frame.DataFrame'>
Index: 2420521 entries, n003bba8a98662e4 to nfff2bd38e397265
Columns: 1222 entries, era to target_thomas_v4_60
dtypes: float32(1220), object(2)
memory usage: 11.1+ GB
The size of the feature is: 1191
The size of the targets is: 29
The size of eras + features + targets = 1221
['target_nomi_v4_20', 'target_nomi_v4_60', 'target_tyler_v4_20', 'target_tyler_v4_60', 'target_victor_v4_20', 'target_victor_v4_60', 'target_ralph_v4_20', 'target_ralph_v4_60', 'target_waldo_v4_20', 'target_waldo_v4_60', 'target_jerome_v4_20', 'target_jerome_v4_60', 'target_janet_v4_20', 'target_janet_v4_60', 'target_ben_v4_20', 'target_ben_v4_60', 'target_alan_v4_20', 'target_alan_v4_60', 'target_paul_v4_20', 'target_paul_v4_60', 'target_george_v4_20', 'target_george_v4_60', 'target_william_v4_20', 'target_william_v4_60', 'target_arthur_v4_20', 'target_arthur_v4_60', 'target_thomas_v4_20', 'target_thomas_v4_60']
num of target: 28
Training data filled finished!

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [46]:
def init_weights(m):
    if type(m) == nn.Linear:
        init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class MultiTargetNN(nn.Module):
    def __init__(self, input_dim, num_layers=11):
        super(MultiTargetNN, self).__init__()
        layer_sizes = [input_dim] + [8192]*(num_layers-2) + [1191, 28]   #list of the size of hidden layers
        self.fcs = nn.ModuleList([nn.Linear(layer_sizes[i], layer_sizes[i+1]) for i in range(num_layers)])
        self.dropouts = nn.ModuleList([nn.Dropout(p=0.3) for _ in range(num_layers-1)])   
        self.fcs.apply(init_weights)

    def forward(self, x):
        for i in range(len(self.fcs)-1):
            x = torch.relu(self.fcs[i](x))
            x = self.dropouts[i](x)
        x = self.fcs[-1](x)
        return x

In [49]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.3)
X_train_tensor = torch.FloatTensor(X)
Y_train_tensor = torch.FloatTensor(y)
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=8)

In [50]:
input_dim = X.shape[1]
print(f"input_dim = {input_dim}")

model = MultiTargetNN(input_dim)
# model_path = "./models/NN_fea_all_v2.pt"
# model.load_state_dict(torch.load(model_path))   #load the model

model.apply(init_weights)
model = model.to(device)    # GPU

criterion = nn.MSELoss()    # Use Mean Squared Error as the loss function for regression tasks
learning_rate = 1e-7
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

num_epochs = 50
model_path = "./models/NN_fea_all_v2_best.pt"
model.train()       # Enable dropout regularization during training
best_loss = 999
best_output = None

print('Training start!!!!!!!!!!!!!!!')
for epoch in range(num_epochs):
 

    predictions = []
    ground_truths = []  
    counter = 0
    loss_count = 0
    if epoch == 0: print("epoch start!!!!!")
    
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        inputs = inputs.to(device)          # GPU
        targets = targets.to(device)        # GPU
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        current_lr = optimizer.param_groups[0]['lr']
        if counter %1000 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, LR: {current_lr}, Loss: {loss.item()}")
        if loss.item() <= best_loss:
            print(f"\n\nEpoch {epoch + 1}/{num_epochs}, LR: {current_lr}, Loss: {loss.item()}")
            best_loss = loss.item()
            best_output = outputs
            torch.save(model.state_dict(), model_path)
            print(f"targets: {targets[0]}")
            print(f"outputs: {best_output[0]}")
            print("model saved!!")
            print("\n\n")
        else:
            loss_count +=1

        if loss_count > 5:
            break
        counter +=1
    # scheduler.step(loss.item()     #for platue learner
    scheduler.step()
torch.save(model.state_dict(), "./models/NN_fea_all_v2.pt")
print(f"targets: {targets}")
print(f"outputs: {best_output}")

input_dim = 1191
Training start!!!!!!!!!!!!!!!
epoch start!!!!!
Epoch 1/50, LR: 1e-07, Loss: 0.33154261112213135


Epoch 1/50, LR: 1e-07, Loss: 0.33154261112213135
targets: tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.7500, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.7500, 0.2500, 0.2500, 0.5000, 0.5000,
        0.5000, 0.5000, 0.2500, 0.2500, 0.3333, 0.5000, 0.5000, 0.8333, 0.1667,
        0.3333], device='cuda:0')
outputs: tensor([-0.0816, -0.0163, -0.0669, -0.0871, -0.0104,  0.0986, -0.1724, -0.2462,
        -0.0287,  0.2306, -0.1139,  0.1603,  0.1698,  0.0064,  0.1369,  0.2777,
         0.0306, -0.0538,  0.0738, -0.0586, -0.0824, -0.1111, -0.3235,  0.4301,
        -0.1440, -0.1739, -0.0137, -0.1534], device='cuda:0',
       grad_fn=<SelectBackward0>)
model saved!!





Epoch 1/50, LR: 1e-07, Loss: 0.33044618368148804
targets: tensor([0.0000, 0.2500, 0.0000, 0.5000, 0.0000, 0.2500, 0.0000, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.0

## Prediction

In [ ]:
del df_validation

In [ ]:
df_validation = pd.read_parquet('./v4/validation.parquet')

In [ ]:
id = df_validation.index
# features = [c for c in df_validation if c.startswith("feature")]     #get all 1191 feature
features = get_targets(700)     #get top 700 features
df_validation = df_validation[features].fillna(0.5).values

In [ ]:
input_dim = df_validation.shape[1]

model = MultiTargetNN(input_dim)
# model_path = "./models/NN_fea_all_v2.pt"
model_path = "./models/NN_fea_all_v2_best.pt"
model.load_state_dict(torch.load(model_path))
model.eval()

df_validation.shape
X_val_tensor = torch.FloatTensor(df_validation)

with torch.no_grad():  # Disable gradient tracking to save memory
    y_pred_tensor = model(X_val_tensor)
    
# Convert the predicted tensor to a numpy array
y_pred = y_pred_tensor.numpy()
y_pred

array([[0.35387158, 0.3580751 , 0.3361066 , ..., 0.34894812, 0.3303181 ,
        0.32695746],
       [0.35406226, 0.35789284, 0.33607253, ..., 0.3490825 , 0.33050779,
        0.3267658 ],
       [0.3536805 , 0.35766247, 0.33556283, ..., 0.34874552, 0.330077  ,
        0.32675466],
       ...,
       [0.35401136, 0.35790557, 0.33591554, ..., 0.34907123, 0.33049196,
        0.3269428 ],
       [0.35418338, 0.35824668, 0.33647075, ..., 0.34941363, 0.33065742,
        0.32704765],
       [0.3537332 , 0.35784543, 0.33594885, ..., 0.34865335, 0.33006465,
        0.32687825]], dtype=float32)

In [ ]:
print(y_pred.shape)
target_pred = y_pred[:,0]   #extract only target for submission

(2436823, 28)


In [ ]:
clean_predict = np.zeros(target_pred.shape)
count = [0,0,0,0,0]
for i in range(len(target_pred)):
    if target_pred[i] <= 0.125:
        clean_predict[i] = 0
        count[0] +=1
    if target_pred[i]  > 0.125 and target_pred[i]  <= 0.375:
        clean_predict[i] = 0.25
        count[1] +=1
    if target_pred[i] > 0.375 and target_pred[i] <= 0.625:
        clean_predict[i] = 0.5
        count[2] +=1
    if target_pred[i] > 0.625 and target_pred[i] <= 0.875:
        clean_predict[i] = 0.75
        count[3] +=1
    if target_pred[i] > 0.875 and target_pred[i] <= 1:
        clean_predict[i] = 1
        count[4] +=1
count

[0, 2436823, 0, 0, 0]

In [ ]:
# valid_gen_and_save(id, target_pred, 'v4_all_nn_v2')
valid_gen_and_save(id, target_pred, 'v4_all_nn_v2_best')